# Requirements for running this notebook:
- Python =>3.6 (using f-Strings)
- PyTorch => 1.0
- scikit-learn
- NumPy
- PyTables
- mt_data.h5 file: You can generate by yourself with this other notebook or [download it](http://ftp.ebi.ac.uk/pub/databases/chembl/blog/pytorch_mtl/mt_data.h5)

## This notebook trains and test a multi-task neural network on ChEMBL data
- It uses a simple shuffled 80/20 train/test split
- Automatically configures the output layer no matter the number of targets in the training data.
- Tries to use GPU if available
- Saves and loads a model to/from a file


In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data as D
import tables as tb
from sklearn.metrics import (matthews_corrcoef, 
                             confusion_matrix, 
                             f1_score, 
                             roc_auc_score,
                             accuracy_score,
                             roc_auc_score)


In [2]:
# set the device to GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set some config values

In [3]:
MAIN_PATH = '.'
DATA_FILE = 'mt_data.h5'
MODEL_FILE = 'chembl_mt.model'
N_WORKERS = 8 # Dataloader workers, prefetch data in parallel to have it ready for the model after each batch train
BATCH_SIZE = 32 # https://twitter.com/ylecun/status/989610208497360896?lang=es
LR = 2 # Learning rate. Big value because of the way we are weighting the targets
N_EPOCHS = 2 # You should train longer!!!

# Set the dataset loaders

Simple 80/20 train/test split for the example

In [4]:
class ChEMBLDataset(D.Dataset):
    
    def __init__(self, file_path):
        self.file_path = file_path
        with tb.open_file(self.file_path, mode='r') as t_file:
            self.length = t_file.root.fps.shape[0]
            self.n_targets = t_file.root.labels.shape[1]
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        with tb.open_file(self.file_path, mode='r') as t_file:
            structure = t_file.root.fps[index]
            labels = t_file.root.labels[index]
        return structure, labels


dataset = ChEMBLDataset(f"{MAIN_PATH}/{DATA_FILE}")
validation_split = .2
random_seed= 42

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

np.random.seed(random_seed)
np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

train_sampler = D.sampler.SubsetRandomSampler(train_indices)
test_sampler = D.sampler.SubsetRandomSampler(test_indices)

# dataloaders can prefetch the next batch if using n workers while
# the model is tranining
train_loader = torch.utils.data.DataLoader(dataset,
                                           batch_size=BATCH_SIZE,
                                           num_workers=N_WORKERS,
                                           sampler=train_sampler)

test_loader = torch.utils.data.DataLoader(dataset, 
                                          batch_size=BATCH_SIZE,
                                          num_workers=N_WORKERS,
                                          sampler=test_sampler)


# Define the model, the optimizer and the loss criterion

In [5]:
class ChEMBLMultiTask(nn.Module):
    """
    Architecture borrowed from: https://arxiv.org/abs/1502.02072
    """
    def __init__(self, n_tasks):
        super(ChEMBLMultiTask, self).__init__()
        self.n_tasks = n_tasks
        self.fc1 = nn.Linear(1024, 2000)
        self.fc2 = nn.Linear(2000, 100)
        self.dropout = nn.Dropout(0.25)

        # add an independet output for each task int the output laer
        for n_m in range(self.n_tasks):
            self.add_module(f"y{n_m}o", nn.Linear(100, 1))
        
    def forward(self, x):
        h1 = self.dropout(F.relu(self.fc1(x)))
        h2 = F.relu(self.fc2(h1))
        out = [torch.sigmoid(getattr(self, f"y{n_m}o")(h2)) for n_m in range(self.n_tasks)]
        return out
    
# create the model, to GPU if available
model = ChEMBLMultiTask(dataset.n_targets).to(device)

# binary cross entropy
# each task loss is weighted inversely proportional to its number of datapoints, borrowed from:
# http://www.bioinf.at/publications/2014/NIPS2014a.pdf
with tb.open_file(f"{MAIN_PATH}/{DATA_FILE}", mode='r') as t_file:
    weights = torch.tensor(t_file.root.weights[:])
    weights = weights.to(device)

criterion = [nn.BCELoss(weight=w) for x, w in zip(range(dataset.n_targets), weights.float())]

# stochastic gradient descend as an optimiser
optimizer = torch.optim.SGD(model.parameters(), LR)


# Train the model
Given the extremely sparse nature of the dataset is difficult to clearly see how the loss is improving after every batch. It looks clearer after several epochs and much more clear when testing :)

In [6]:
# model is by default in train mode. Training can be resumed after .eval() but needs to be set to .train() again
model.train()
for ep in range(N_EPOCHS):
    for i, (fps, labels) in enumerate(train_loader):
        # move it to GPU if available
        fps, labels = fps.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(fps)
        
        # calc the loss
        loss = torch.tensor(0.0).to(device)
        for j, crit in enumerate(criterion):
            # mask keeping labeled molecules for each task
            mask = labels[:, j] >= 0.0
            if len(labels[:, j][mask]) != 0:
                # the loss is the sum of each task/target loss.
                # there are labeled samples for this task, so we add it's loss
                loss += crit(outputs[j][mask], labels[:, j][mask].view(-1, 1))

        loss.backward()
        optimizer.step()

        if (i+1) % 500 == 0:
            print(f"Epoch: [{ep+1}/{N_EPOCHS}], Step: [{i+1}/{len(train_indices)//BATCH_SIZE}], Loss: {loss.item()}")
    

Epoch: [1/2], Step: [500/17789], Loss: 0.01780553348362446
Epoch: [1/2], Step: [1000/17789], Loss: 0.01136045902967453
Epoch: [1/2], Step: [1500/17789], Loss: 0.018664617091417313
Epoch: [1/2], Step: [2000/17789], Loss: 0.013626799918711185
Epoch: [1/2], Step: [2500/17789], Loss: 0.012855792418122292
Epoch: [1/2], Step: [3000/17789], Loss: 0.013796127401292324
Epoch: [1/2], Step: [3500/17789], Loss: 0.021601887419819832
Epoch: [1/2], Step: [4000/17789], Loss: 0.00950919184833765
Epoch: [1/2], Step: [4500/17789], Loss: 0.02028888650238514
Epoch: [1/2], Step: [5000/17789], Loss: 0.013251284137368202
Epoch: [1/2], Step: [5500/17789], Loss: 0.008788244798779488
Epoch: [1/2], Step: [6000/17789], Loss: 0.012066680938005447
Epoch: [1/2], Step: [6500/17789], Loss: 0.013928443193435669
Epoch: [1/2], Step: [7000/17789], Loss: 0.011484757997095585
Epoch: [1/2], Step: [7500/17789], Loss: 0.0071386718191206455
Epoch: [1/2], Step: [8000/17789], Loss: 0.014712771400809288
Epoch: [1/2], Step: [8500/17

# Test the model

In [7]:
y_trues = []
y_preds = []
y_preds_proba = []

# do not track history
with torch.no_grad():
    for fps, labels in test_loader:
        # move it to GPU if available
        fps, labels = fps.to(device), labels.to(device)
        # set model to eval, so will not use the dropout layer
        model.eval()
        outputs = model(fps)
        for j, out in enumerate(outputs):
            mask = labels[:, j] >= 0.0
            y_pred = torch.where(out[mask] > 0.5, torch.ones(1), torch.zeros(1)).view(1, -1)

            if y_pred.shape[1] > 0:
                for l in labels[:, j][mask].long().tolist():
                    y_trues.append(l)
                for p in y_pred.view(-1, 1).tolist():
                    y_preds.append(int(p[0]))
                for p in out[mask].view(-1, 1).tolist():
                    y_preds_proba.append(float(p[0]))

tn, fp, fn, tp = confusion_matrix(y_trues, y_preds).ravel()
sens = tp / (tp + fn)
spec = tn / (tn + fp)
prec = tp / (tp + fp)
f1 = f1_score(y_trues, y_preds)
acc = accuracy_score(y_trues, y_preds)
mcc = matthews_corrcoef(y_trues, y_preds)
auc = roc_auc_score(y_trues, y_preds_proba)

print(f"accuracy: {acc}, auc: {auc}, sens: {sens}, spec: {spec}, prec: {prec}, mcc: {mcc}, f1: {f1}")
print(f"Not bad for only {N_EPOCHS} epochs!")

accuracy: 0.8371918235997756, auc: 0.8942389411754185, sens: 0.7053822792666977, spec: 0.8987519347341067, prec: 0.7649158653846154, mcc: 0.6179805824644773, f1: 0.733943790291889
Not bad for only 2 epochs!


# Save the model to a file

In [8]:
torch.save(model.state_dict(), f"./{MODEL_FILE}")

# Load the model from the file

In [9]:
model = ChEMBLMultiTask(560) # number of tasks
model.load_state_dict(torch.load(f"./{MODEL_FILE}"))
model.eval()

ChEMBLMultiTask(
  (fc1): Linear(in_features=1024, out_features=2000, bias=True)
  (fc2): Linear(in_features=2000, out_features=100, bias=True)
  (dropout): Dropout(p=0.25)
  (y0o): Linear(in_features=100, out_features=1, bias=True)
  (y1o): Linear(in_features=100, out_features=1, bias=True)
  (y2o): Linear(in_features=100, out_features=1, bias=True)
  (y3o): Linear(in_features=100, out_features=1, bias=True)
  (y4o): Linear(in_features=100, out_features=1, bias=True)
  (y5o): Linear(in_features=100, out_features=1, bias=True)
  (y6o): Linear(in_features=100, out_features=1, bias=True)
  (y7o): Linear(in_features=100, out_features=1, bias=True)
  (y8o): Linear(in_features=100, out_features=1, bias=True)
  (y9o): Linear(in_features=100, out_features=1, bias=True)
  (y10o): Linear(in_features=100, out_features=1, bias=True)
  (y11o): Linear(in_features=100, out_features=1, bias=True)
  (y12o): Linear(in_features=100, out_features=1, bias=True)
  (y13o): Linear(in_features=100, out_featur